# Objective: Convolutional Neural Network Classification Using CIFAR-10
Here, I'm going to build an convolutional neural network classification model using PyTorch using CIFAR-10 training data set.

## Import Libraries

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## Load Data
This covers how to download the data and load the data into the code.

In [2]:
# load train dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='.',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

100.0%


Extracting ./cifar-10-python.tar.gz to .


In [5]:
# analyze train dataset
train_dataset.data.shape
set(train_dataset.targets)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [8]:
# load test dataset
test_dataset = torchvision.datasets.CIFAR10(
    root='.',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

Files already downloaded and verified


In [9]:
# analyze test dataset
test_dataset.data.shape

(10000, 32, 32, 3)

## Preprocess Data
This covers how to create batch using DataLoader.

In [11]:
# get the number of classes
K = len(set(train_dataset.targets))
print(f"The number of classes: {K}")

The number of classes: 10


In [13]:
# dataloader to automatically generate batches in the training loop with shuffling
batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [15]:
# check whether the dataloader works okay or not
tmp_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size=1,
                                         shuffle=True)
for x, y in tmp_loader:
    print(x)
    print(x.shape)
    # [batch_size, num_channels, image_height, image_width]
    break

tensor([[[[0.4157, 0.4157, 0.4235,  ..., 0.1569, 0.1333, 0.1176],
          [0.4627, 0.4588, 0.4784,  ..., 0.1412, 0.1255, 0.1216],
          [0.5294, 0.5216, 0.5176,  ..., 0.1451, 0.1255, 0.1216],
          ...,
          [0.7961, 0.8196, 0.8471,  ..., 0.9451, 0.9412, 0.9333],
          [0.8078, 0.8353, 0.8627,  ..., 0.9098, 0.9137, 0.8941],
          [0.8510, 0.8549, 0.8784,  ..., 0.9098, 0.8980, 0.8824]],

         [[0.3608, 0.3569, 0.3608,  ..., 0.0784, 0.0667, 0.0627],
          [0.4000, 0.3922, 0.4078,  ..., 0.0824, 0.0667, 0.0627],
          [0.4549, 0.4431, 0.4392,  ..., 0.0824, 0.0667, 0.0667],
          ...,
          [0.7412, 0.7725, 0.8078,  ..., 0.9529, 0.9412, 0.9294],
          [0.7529, 0.7882, 0.8235,  ..., 0.9137, 0.9020, 0.8784],
          [0.7961, 0.8118, 0.8392,  ..., 0.9098, 0.8863, 0.8588]],

         [[0.2980, 0.2824, 0.2745,  ..., 0.0667, 0.0471, 0.0392],
          [0.3373, 0.3176, 0.3216,  ..., 0.0627, 0.0471, 0.0431],
          [0.3922, 0.3647, 0.3490,  ..., 0

## Build Model
This covers how to build a convolutional neural network classification model using fashion MNIST dataset to train. Since the dataset is big enough, we can use GPU for faster process if available.

In [16]:
# define the model
class CNN(nn.Module):
    def __init__(self, K):
        super(CNN, self).__init__()
        
        # define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=2)
        # H_out = floor[(H_in + 2*p - d(k-1) - 1)/2 + 1]
        # image size: 32X32 -> 15X15
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2)
        # image size: 15X15 -> 7X7
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2)
        # image size: 7X7 -> 3X3
        
        # define linear (fc) layers
        self.fc1 = nn.Linear(128 * 3 * 3, 1024)
        self.fc2 = nn.Linear(1024, K)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.relu(self.conv2(X))
        X = F.relu(self.conv3(X))
        X = X.view(-1, 128 * 3 * 3)
        X = F.dropout(X, p=0.5)
        X = F.relu(self.fc1(X))
        X = F.dropout(X, p=0.2)
        return self.fc2(X)

In [17]:
# instantiate the model
model = CNN(K)

In [18]:
# activate gpu if possible, otherwise cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# send the model to the device
model.to(device)


cuda:0


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (fc1): Linear(in_features=1152, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [ ]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())